In [ ]:
# Importing Necessary Libraries
import os
import nltk
import math
import random
from collections import Counter
from collections import defaultdict


In [ ]:
# Tokenizer to keep only Alpha-Numeric Terms  [a-zA-Z0-9_]
tokenizer = nltk.RegexpTokenizer(r"\w+")

global corpus
corpus = {}

global index
index = defaultdict(list)

def PreProcess(data):
    data = ' '.join(tokenizer.tokenize(data))
    data = data.lower()
    return data

In [ ]:
# Function to Calculate Freequency of All Terms in Entire Corpus
def term_freq_corpus(corpus):
    term_freq_c = Counter({})
    for doc in corpus:
        term_freq_c += Counter(corpus[doc])
    return term_freq_c

In [ ]:
# Function to Calculate TF,IDF Values of a Term
def tfidf_val_calc(term,term_indxs_lst,ln_corpus,term_freq_c):
    # tf : Term Freequency of Term in Document, rounded off to 8 decimal places
    try:
        tf = round(((1 + math.log10(len(term_indxs_lst[term])))),8)
    except:
        tf = 0
    # idf : Inverse Document Freequency of Term in Corpus, rounded off to 8 decimal places
    try:
        idf = round((math.log10(ln_corpus / term_freq_c[term])),8)
    except:
        idf = 0
    return tf,idf